In [1]:

!pip install laserembeddings==1.1.2
#!python -m laserembeddings download-models
import torch
from tqdm.auto import tqdm
import re
tqdm.pandas()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from laserembeddings import Laser
import site
import sys
import os
import gdown
import shutil
pd.set_option('display.max_colwidth', None)
#os.system('python -m laserembeddings download-models')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 859 kB 6.4 MB/s 
     |████████████████████████████████| 45 kB 4.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=7dc30b85c3371d2993e7637db39c0442bb40cca99a5680f05a6e0bba773ed38e
  Stored in directory: /root/.cache/pip/wheels/d1/ff/0e/e00ff1e22100702ac8b24e709551ae0fb29db9ffc843510a64
Successfully built sacremoses


In [2]:
torch.__version__

'1.12.1+cu113'

In [3]:
torch.cuda.is_available()

True

In [4]:

loc = site.getsitepackages() 
data_path = loc[0]+'/laserembeddings/data'
print(site.getusersitepackages()) # String for user-specific package location)
loc[0]

/root/.local/lib/python3.7/site-packages


'/usr/local/lib/python3.7/dist-packages'

In [5]:

# assets folder
url = "https://drive.google.com/drive/folders/1gMDdBPFYcc0ACCVZr5tJ82JoaN-Le0aR?usp=sharing"
id = "1gMDdBPFYcc0ACCVZr5tJ82JoaN-Le0aR"

In [6]:
%%time

dir = './data_driven_theology/'
if not os.path.exists(dir):
    os.mkdir(dir)

files = os.listdir(dir)
if(len(files)<5):
    print("downloading necessary files....")
    gdown.download_folder(id=id, quiet=True, use_cookies=False)    

    shutil.copy('/content/data_driven_theology/93langs.fcodes', data_path)
    shutil.copy('/content/data_driven_theology/93langs.fvocab', data_path)
    shutil.copy('/content/data_driven_theology/bilstm.93langs.2018-12-26.pt', data_path)


downloading necessary files....
CPU times: user 681 ms, sys: 663 ms, total: 1.34 s
Wall time: 10.2 s


In [7]:
def similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)
    return np.dot(v1, v2) / n1 / n2

def pairwise_euclidean_dists(x, y):
    dists = -2 * np.matmul(x, y.T)
    dists +=  np.sum(x**2, axis=1)[:, np.newaxis]
    dists += np.sum(y**2, axis=1)
    return  np.sqrt(dists)


laser = Laser()

In [8]:
sim_quran_torah = pd.read_csv('./data_driven_theology/ensemble_preds_most_sim_pairs_quran_and_old_testament.csv')
corpus_emb = np.load('./data_driven_theology/mlt_quran_torah_emb.npy')
sim_quran_torah.head(1)

,t_citation,t_book,t_chapter,t_verse,t_text,q_Name,q_Surah,q_Ayat,q_Verse,similarity_score
0,Jeremiah 51:15,Jeremiah,51,15,"He hath made the earth by his power, he hath established the world by his wisdom, and hath stretched out the heaven by his understanding. \n",The Bee,16,3,He hath created the heavens and the earth with truth. High be He Exalted above all that they associate (with Him).,93


In [9]:
def Multilingual_Quran_Bible_Search_Engine(query,size=10,language = 'en',metric = 'dot'):

    query_embedding = laser.embed_sentences(query, lang=language)

    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = similarity(corpus_emb, query_embedding)
    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]
        Top_index_doc = Top_index_doc[:size]

  
    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'t_citation'] = str(sim_quran_torah['t_citation'][index])
        find.loc[i,'t_book'] = str(sim_quran_torah['t_book'][index])
        find.loc[i,'t_chapter'] = str(sim_quran_torah['t_chapter'][index])
        find.loc[i,'t_verse'] = str(sim_quran_torah['t_verse'][index]) 
        find.loc[i,'t_text'] = str(sim_quran_torah['t_text'][index]) 
        
        find.loc[i,'q_Name'] = str(sim_quran_torah['q_Name'][index])
        find.loc[i,'q_Surah'] = str(sim_quran_torah['q_Surah'][index])
        find.loc[i,'q_Ayat'] = str(sim_quran_torah['q_Ayat'][index])
        find.loc[i,'q_Verse'] = str(sim_quran_torah['q_Verse'][index])
        find.loc[i,'similarity_score'] = str(sim_quran_torah['similarity_score'][index])

    for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
        find.loc[j,'Score'] = simScore
  
    return find

In [10]:
%%time
result = Multilingual_Quran_Bible_Search_Engine("what is calamity?",size=2,language = 'en',metric = 'l2')

CPU times: user 577 ms, sys: 88.1 ms, total: 665 ms
Wall time: 1.42 s


In [11]:
result

,t_citation,t_book,t_chapter,t_verse,t_text,q_Name,q_Surah,q_Ayat,q_Verse,similarity_score,Score
0,Proverbs 24:22,Proverbs,24,22,For their calamity shall rise suddenly; and who knoweth the ruin of them both? \n,The Calamity,101,3,"Ah, what will convey unto thee what the Calamity is!",71,0.801299
1,Song of Solomon 2:6,Song of Solomon,2,6,"His left hand is under my head, and his right hand doth embrace me. \n",The Event,56,9,And (then) those on the left hand; what of those on the left hand ?,76,0.759881


In [12]:
%%time

result = Multilingual_Quran_Bible_Search_Engine("খারাপ জিনিস",size=5,language = 'bn',metric = 'l2')
result

CPU times: user 57.1 ms, sys: 28.9 ms, total: 86.1 ms
Wall time: 95.4 ms


,t_citation,t_book,t_chapter,t_verse,t_text,q_Name,q_Surah,q_Ayat,q_Verse,similarity_score,Score
0,Proverbs 24:22,Proverbs,24,22,For their calamity shall rise suddenly; and who knoweth the ruin of them both? \n,The Calamity,101,3,"Ah, what will convey unto thee what the Calamity is!",71,0.808398
1,Psalms 37:38,Psalms,37,38,But the transgressors shall be destroyed together: the end of the wicked shall be cut off. \n,Sâd,38,55,"This (is for the righteous). And lo! for the transgressors there with be an evil journey’s end,",71,0.803452
2,Proverbs 21:18,Proverbs,21,18,"The wicked shall be a ransom for the righteous, and the transgressor for the upright. \n",Sâd,38,55,"This (is for the righteous). And lo! for the transgressors there with be an evil journey’s end,",71,0.781635
3,Psalms 35:12,Psalms,35,12,They rewarded me evil for good to the spoiling of my soul. \n,The Tidings,78,26,Reward proportioned (to their evil deeds).,76,0.780270
4,Numbers 19:22,Numbers,19,22,And whatsoever the unclean person toucheth shall be unclean; and the soul that toucheth it shall be unclean until even. \n,The Event,56,79,"Which none toucheth save the purified,",71,0.741315
